In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 100)
plt.style.use("seaborn-whitegrid")

In [3]:
data = pd.read_csv("data.csv")

In [4]:
data.head(3)

,Category,City,Country/Region,Customer Name,Manufacturer,Order Date,Order ID,Postal Code,Product Name,Region,Segment,Ship Date,Ship Mode,State/Province,Sub-Category,Discount,Profit,Quantity,Sales
0,Office Supplies,Houston,United States,Darren Powers,Message Book,1/3/2019,US-2019-103800,77095,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",Central,Consumer,1/7/2019,Standard Class,Texas,Paper,0.2,5.5512,2,16.448
1,Office Supplies,Naperville,United States,Phillina Ober,GBC,1/4/2019,US-2019-112326,60540,GBC Standard Plastic Binding Systems Combs,Central,Home Office,1/8/2019,Standard Class,Illinois,Binders,0.8,-5.4870,2,3.540
2,Office Supplies,Naperville,United States,Phillina Ober,Avery,1/4/2019,US-2019-112326,60540,Avery 508,Central,Home Office,1/8/2019,Standard Class,Illinois,Labels,0.2,4.2717,3,11.784


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        10194 non-null  object 
 1   City            10194 non-null  object 
 2   Country/Region  10194 non-null  object 
 3   Customer Name   10194 non-null  object 
 4   Manufacturer    10194 non-null  object 
 5   Order Date      10194 non-null  object 
 6   Order ID        10194 non-null  object 
 7   Postal Code     10194 non-null  object 
 8   Product Name    10194 non-null  object 
 9   Region          10194 non-null  object 
 10  Segment         10194 non-null  object 
 11  Ship Date       10194 non-null  object 
 12  Ship Mode       10194 non-null  object 
 13  State/Province  10194 non-null  object 
 14  Sub-Category    10194 non-null  object 
 15  Discount        10194 non-null  float64
 16  Profit          10194 non-null  float64
 17  Quantity        10194 non-null 

In [6]:
data.isnull().sum()

Category          0
City              0
Country/Region    0
Customer Name     0
Manufacturer      0
Order Date        0
Order ID          0
Postal Code       0
Product Name      0
Region            0
Segment           0
Ship Date         0
Ship Mode         0
State/Province    0
Sub-Category      0
Discount          0
Profit            0
Quantity          0
Sales             0
dtype: int64

In [7]:
# Convert to datetime

data["Order Date"] = pd.to_datetime(data["Order Date"])

data["Ship Date"] = pd.to_datetime(data["Ship Date"])

In [8]:
data.dtypes

Category                  object
City                      object
Country/Region            object
Customer Name             object
Manufacturer              object
Order Date        datetime64[ns]
Order ID                  object
Postal Code               object
Product Name              object
Region                    object
Segment                   object
Ship Date         datetime64[ns]
Ship Mode                 object
State/Province            object
Sub-Category              object
Discount                 float64
Profit                   float64
Quantity                   int64
Sales                    float64
dtype: object

- Data cleaned, ready to use

# **RFM Analysis**

Calculate Recency Score

In [9]:
# set current date = Max(Order Date) + 1

import datetime


current_date = max(data["Order Date"]) + datetime.timedelta(days = 1)

In [10]:
current_date

Timestamp('2022-12-31 00:00:00')

# Data 2022

In [11]:
data_2022 = data.loc[data['Order Date'] >= '2022-01-01', :].copy()

In [12]:
# Find the last active day

rfm_data_2022 = data_2022.groupby(["Customer Name"]).agg(
                                                {'Order Date': lambda x: (current_date - x.max()).days,
                                                'Order ID': 'count',
                                                'Sales': 'sum'
                                                }
                                            )

In [13]:
rfm_data_2022.columns = ("recency", "frequency", "monetary")

In [14]:
rfm_data_2022['r_percentile_rank'] = rfm_data_2022.recency.rank(pct = True, ascending = False)
rfm_data_2022['f_percentile_rank'] = rfm_data_2022.frequency.rank(pct = True, ascending = True)
rfm_data_2022['m_percentile_rank'] = rfm_data_2022.monetary.rank(pct = True, ascending = True)

In [15]:
from contextlib import nullcontext


def scoring(x):
    score = 0
    
    if x < 0.2:
        score = 1
    elif x >= 0.2 and x < 0.4:
        score = 2
    elif x >= 0.4 and x < 0.6:
        score = 3
    elif x >= 0.6 and x < 0.8:
        score = 4
    elif x >= 0.8 and x <= 1:
        score = 5

    return score

In [16]:
rfm_data_2022['r_score'] = rfm_data_2022['r_percentile_rank'].apply(scoring)
rfm_data_2022['f_score'] = rfm_data_2022['f_percentile_rank'].apply(scoring)
rfm_data_2022['m_score'] = rfm_data_2022['m_percentile_rank'].apply(scoring)


In [17]:
rfm_data_2022['rfm_score'] = rfm_data_2022['r_score'].astype(str)\
                             + rfm_data_2022['f_score'].astype(str)\
                                 + rfm_data_2022['m_score'].astype(str)

<img src="rfm_segments_table.png" width="1000px">

In [18]:
from asyncio.windows_events import NULL
from msilib import AMD64


def customer_rfm_classification(x):
    customer = ''

    if x[0] in ('4','5') and x[1] in ('4','5') and x[2] in ('4','5'):
        customer = 'Champions'

    elif x[0] in ('4','5') and x[1] in ('1','2','3') and x[2] in ('4','5'):
        customer = 'Recent Wholesale'

    elif x[0] in ('1','2','3') and x[1] in ('1','2','3') and x[2] in ('4','5'):
        customer = 'Old Wholesale'

    elif x[0] in ('4','3','5') and x[1] in ('4','3') and x[2] in ('4','5'):
        customer = 'Loyal Customers'

    elif x[0] in ('4','3','5') and x[1] in ('5','4','3') and x[2] in ('3','2','1'):
        customer = 'Loyal Customers but Small Spenders'

    elif x[0] in ('5','4','3') and x[1] in ('1','3','2') and x[2] in ('1','2','3'):
        customer = 'Potential Loyalists'
    
    elif x[0] in ('4','5') and x[1] in ('1') and x[2] in ('1'):
        customer = 'New Customers'
    
    elif x[0] in ('4','3') and x[1] in ('1') and x[2] in ('1'):
        customer = 'Promising'
    
    elif x[0] in ('4','3') and x[1] in ('4','3') and x[2] in ('4','3'):
        customer = 'Need Attention'
    
    elif x[0] in ('2','3') and x[1] in ('2','1') and x[2] in ('2','1'):
        customer = 'About to Sleep'
    
    elif x[0] in ('3','2','1') and x[1] in ('4','3','2','5') and x[2] in ('2','3','4','5'):
        customer = 'At risk'
    
    elif x[0] in ('1','2') and x[1] in ('4','5') and x[2] in ('4','5'):
        customer = "Can't Lose Them"
    
    elif x[0] in ('2','3') and x[1] in ('1','2','3','4') and x[2] in ('1','2','3'):
        customer = 'Hibernating'
    
    elif x[0] in ('1','2') and x[1] in ('1','2','3') and x[2] in ('1','2','3'):
        customer = 'Lost'
    
    else:
        customer = NULL

    return customer

In [19]:
rfm_data_2022['customer_segmentation'] = rfm_data_2022['rfm_score'].apply(customer_rfm_classification)

In [20]:
rfm_data_2022['customer_segmentation'].value_counts(normalize= True)

At risk                               0.220000
Loyal Customers but Small Spenders    0.164286
Potential Loyalists                   0.150000
Champions                             0.137143
Old Wholesale                         0.092857
Lost                                  0.075714
About to Sleep                        0.075714
Recent Wholesale                      0.064286
Loyal Customers                       0.015714
Hibernating                           0.004286
Name: customer_segmentation, dtype: float64

# DATA 2021

In [21]:
data_2021 = data.loc[data['Order Date'].dt.strftime('%Y') == '2021'].copy()

In [22]:
# Find the last active day

rfm_data_2021 = data_2021.groupby(["Customer Name"]).agg(
                                                {'Order Date': lambda x: (current_date - x.max()).days,
                                                'Order ID': 'count',
                                                'Sales': 'sum'
                                                }
                                            )

In [23]:
rfm_data_2021.columns = ("recency", "frequency", "monetary")

In [24]:
rfm_data_2021['r_percentile_rank'] = rfm_data_2021.recency.rank(pct = True, ascending = False)
rfm_data_2021['f_percentile_rank'] = rfm_data_2021.frequency.rank(pct = True, ascending = True)
rfm_data_2021['m_percentile_rank'] = rfm_data_2021.monetary.rank(pct = True, ascending = True)

In [25]:
rfm_data_2021['r_score'] = rfm_data_2021['r_percentile_rank'].apply(scoring)
rfm_data_2021['f_score'] = rfm_data_2021['f_percentile_rank'].apply(scoring)
rfm_data_2021['m_score'] = rfm_data_2021['m_percentile_rank'].apply(scoring)

In [26]:
rfm_data_2021['rfm_score'] = rfm_data_2021['r_score'].astype(str)\
                             + rfm_data_2021['f_score'].astype(str)\
                                 + rfm_data_2021['m_score'].astype(str)

In [27]:
rfm_data_2021['customer_segmentation'] = rfm_data_2021['rfm_score'].apply(customer_rfm_classification)

In [28]:
rfm_data_2021['customer_segmentation'].value_counts()

At risk                               132
Champions                             109
Loyal Customers but Small Spenders    105
Potential Loyalists                    99
Lost                                   62
About to Sleep                         46
Old Wholesale                          46
Recent Wholesale                       30
Loyal Customers                        19
Hibernating                             2
Name: customer_segmentation, dtype: int64

In [29]:
rfm_data_2022['customer_segmentation'].value_counts()

At risk                               154
Loyal Customers but Small Spenders    115
Potential Loyalists                   105
Champions                              96
Old Wholesale                          65
Lost                                   53
About to Sleep                         53
Recent Wholesale                       45
Loyal Customers                        11
Hibernating                             3
Name: customer_segmentation, dtype: int64

In [30]:
rfm_data_2022['customer_segmentation'].value_counts(normalize= True)*100

At risk                               22.000000
Loyal Customers but Small Spenders    16.428571
Potential Loyalists                   15.000000
Champions                             13.714286
Old Wholesale                          9.285714
Lost                                   7.571429
About to Sleep                         7.571429
Recent Wholesale                       6.428571
Loyal Customers                        1.571429
Hibernating                            0.428571
Name: customer_segmentation, dtype: float64

In [31]:
rfm_data_2021['customer_segmentation'].value_counts(normalize= True)*100

At risk                               20.307692
Champions                             16.769231
Loyal Customers but Small Spenders    16.153846
Potential Loyalists                   15.230769
Lost                                   9.538462
About to Sleep                         7.076923
Old Wholesale                          7.076923
Recent Wholesale                       4.615385
Loyal Customers                        2.923077
Hibernating                            0.307692
Name: customer_segmentation, dtype: float64

- Champions giảm kha khá
- Old Wholsale tăng
- Lost tăng
- Recent Wholesale tăng